$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\cset}[1]{\mathcal{#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bb}[1]{\boldsymbol{#1}}
\newcommand{\E}[2][]{\mathbb{E}_{#1}\left[#2\right]}
\newcommand{\ip}[3]{\left<#1,#2\right>_{#3}}
\newcommand{\given}[]{\,\middle\vert\,}
\newcommand{\DKL}[2]{\cset{D}_{\text{KL}}\left(#1\,\Vert\, #2\right)}
\newcommand{\grad}[]{\nabla}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
$$

# Part 2: Summary Questions
<a id=part2></a>

This section contains summary questions about various topics from the course material.

You can add your answers in new cells below the questions.

**Notes**

- Clearly mark where your answer begins, e.g. write "**Answer:**" in the beginning of your cell.
- Provide a full explanation, even if the question doesn't explicitly state so. We will reduce points for partial explanations!
- This notebook should be runnable from start to end without any errors.

### CNNs

1. Explain the meaning of the term "receptive field" in the context of CNNs.

------------
ANSWER:

The receptive field is the region in the input space, which is observable by some CNN feature. 
Expliciply - which features in more previous layers do affect some specific feature in later layer. 

------------

2. Explain and elaborate about three different ways to control the rate at which the receptive field grows from layer to layer. Compare them to each other in terms of how they combine input features.

------------

ANSWER:

The three ways are:
1. Increase the kernel size of a filter. E.g. from 3X3 to 5X5. This way we directly increase the amount of features that influence 1 feature in the next layer.
2. Increase the stride of the filter. This technique will 'skip' one feature in the filter, thus in the end, bigger amount of features influence the feature that we want to increase its receptive field. This technique is widely used to increase the receptive field dramatically, and not to be focused locally on 1 spot, increasing the spatial extent. Important if we want to focus on large parts of image, and not small.
3. Pooling is a an effective way to increase receptive field. By, for example, making 2x2 pooling (max/avg), each feature combines the information of 4 features from a previous layer, and consequently, combines the information that each of those features had.

In practice, all of those methods are widely used in CNNs.


------------

3. Imagine a CNN with three convolutional layers, defined as follows:

In [1]:
import torch
import torch.nn as nn

cnn = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=4, out_channels=16, kernel_size=5, stride=2, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=7, dilation=2, padding=3),
    nn.ReLU(),
)

cnn(torch.rand(size=(1, 3, 1024, 1024), dtype=torch.float32)).shape

torch.Size([1, 32, 122, 122])

What is the size (spatial extent) of the receptive field of each "pixel" in the output tensor?

------------

ANSWER:

Calculating (from end to start):
1. In last layer the kernel size is 7, with dilation of 2. Local receptive field = 13
2. Maxpool2d is used with kernel size of 2. Receptive field increased to 26.
3. Conv2d with k=5, s=2. Receptive field increases to 52 (from stride) + 4 (from kernel) = 56
4. Another maxpool2d with padding of 2. Increase to 112.
5. Conv2d with kernel size of 3. Increase to 114.

------------

4. You have trained a CNN, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$, and $f_l(\cdot;\vec{\theta}_l)$ is a convolutional layer (not including the activation function).

  After hearing that residual networks can be made much deeper, you decide to change each layer in your network you used the following residual mapping instead $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)+\vec{x}$, and re-train.

  However, to your surprise, by visualizing the learned filters $\vec{\theta}_l$ you observe that the original network and the residual network produce completely different filters. Explain the reason for this.

------------

ANSWER:

What is missing is of course the non-linear part, e.g. the activation function which is used in the residual block. By simply summing $x$ to the output from the filter, is like adding another filter operation with kernel of size 1x1, and weight of 1. And as we know, applying filters is a linear operation, which can be replaced with 1 filter. So, $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)+\vec{x}$  would be a wrong thing to do.
Instead, we have to introduce an activation function after the convolution layer. Then, sum with $x$ and pass the result to the next activation layer. This is known as a ResNet block (basic one, not bottleneck)


------------

### Dropout

1. Consider the following neural network:

In [2]:
import torch.nn as nn

p1, p2 = 0.1, 0.2
nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Dropout(p=p1),
    nn.Dropout(p=p2),
)

Sequential(
  (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Dropout(p=0.1, inplace=False)
  (3): Dropout(p=0.2, inplace=False)
)

If we want to replace the two consecutive dropout layers with a single one defined as follows:
```python
nn.Dropout(p=q)
```
what would the value of `q` need to be? Write an expression for `q` in terms of `p1` and `p2`.

------------

ANSWER:

The parameter 'p' in the Dropout layer is the probability that a certain element will be 'zeroed'. 
This means, with probability of $1-p$, a neuron will stay.

For first layer, $1-p_1 = 0.9$

For second layer, $1-p_2 = 0.8$

This means the neuron will stay after 2 those layers with probability of $0.9 \cdot 0.8 = 0.72$
Hence, it will dropout with probability of 0.28

So: $q = 0.28$

------------

2. **True or false**: dropout must be placed only after the activation function.

------------

ANSWER:
FALSE

This depends on the activation function first of all. If we use ReLU, like we often do, this doesn't matter, since anyway, 0 maps to 0 in ReLU.

------------

3. After applying dropout with a drop-probability of $p$, the activations are scaled by $1/(1-p)$. Prove that this scaling is required in order to maintain the value of each activation unchanged in expectation.

------------

ANSWER:

We do the scaling because in the Test phase, dropout is not applied, and all the neurons are active.
So we have to keep the output of each neuron at test time equal to expected output in the training time.
Let's denote output of a neuron as $x$ and dropout probability as $p$

Without scaling:

$E(x) = (1-p)\cdot x + p\cdot0 = (1-p)\cdot x$

Scaling by $1/(1-p)$ gives:

$E(x) = ((1-p)\cdot x + p\cdot0)\cdot (1/(1-p))  = ((1-p)\cdot x) \cdot (1/(1-p)) = x$

$E(x) = x$

So we maintain the original value unchanged

------------

### Losses and Activation functions

1. You're training a an image classifier that, given an image, needs to classify it as either a dog (output 0) or a hotdog (output 1). Would you train this model with an L2 loss? if so, why? if not, demonstrate with a numerical example. What would you use instead?

------------

ANSWER:

This is clearly a classification task

Why L2 Loss is a bad choice:
The scores given in the question certainly show that those are categorical values. 

the output of the network is unbounded. Meaning that giving output of 3, I certainly believe more that this is a hotdog rathat then a dog, but my loss will still be high, and I will correct the neurons for giving 'wrong' result.

What I would do:

For categorical tasks we usually use the Cross Entropy loss function, which maps the output to probabilities of a certain class to be the category.
I would use this Cross Entropy Loss, and change the output to give me **2** outputs. Then I will use the Categorical Cross Entropy Loss (Softmax + CE Loss).


------------

2. After months of research into the origins of climate change, you observe the following result:

<center><img src="https://sparrowism.soc.srcf.net/home/piratesarecool4.gif" /></center>

You decide to train a cutting-edge deep neural network regression model, that will predict the global temperature based on the population of pirates in `N` locations around the globe.
You define your model as follows:

In [3]:
import torch.nn as nn

N = 42  # number of known global pirate hot spots
H = 128
mlpirate = nn.Sequential(
    nn.Linear(in_features=N, out_features=H),
    nn.Sigmoid(),
    *[
        nn.Linear(in_features=H, out_features=H), nn.Sigmoid(),
    ]*24,
    nn.Linear(in_features=H, out_features=1),
)

While training your model you notice that the loss reaches a plateau after only a few iterations.
It seems that your model is no longer training.
What is the most likely cause?

------------

ANSWER:

We can notice that there are 24 pairs of 'Linear - Sigmoid' blocks used. This will certainly cause the problem of vanishing gradients in the network, because gradients are being multiplied as they progagate into the network during the backpropagation process. In sigmoid function the gradients are close to zero when the absolute value of the input is big. This will cause the deeper layers to get really small updates to their weights, thus not to learn.

------------

3. Referring to question 2 above: A friend suggests that if you replace the `sigmoid` activations with `tanh`, it will solve your problem. Is he correct? Explain why or why not.

------------

ANSWER:

Tanh will **not** solve the problem, since its gradients are also small when the input absolute values are high.

What can solve (mitigate) are for example:
1. ReLU activation
2. ResNet architecture

------------

4. Regarding the ReLU activation, state whether the following sentences are **true or false** and explain:
  1. In a model using exclusively ReLU activations, there can be no vanishing gradients.
  1. The gradient of ReLU is linear with its input when the input is positive.
  1. ReLU can cause "dead" neurons, i.e. activations that remain at a constant value of zero.

------------

ANSWER:

1. FALSE

ReLU doesn't completely solve the vanishing gradient problem (otherwise, the main reason for ResNet block would not exist, we could simply use ReLU). What is being propagated backwards are also the weights of the neurons, and if those are below 1 in absolute value, gradients will vanish anyway.

2. FALSE

The gradient of ReLU is 1 if input is positive and 0 if it's negative.

3. TRUE

This is known as the "Dead RELU" problem. When the neuron is at this state, nothing can take him out, since its output is always 0 from now on, and no gradient will go backwards to fix the weights. The possible fix is using the Leaky ReLU.

------------

### Optimization

1. Explain the difference between: stochastic gradient descent (SGD), mini-batch SGD and regular gradient descent (GD).

------------

ANSWER:

- GD: updating the weights in the gradient direction, and the gradient is calculated by using ALL input data (training set)
- SGD: using single example from the input space to update the gradient. e.g. updating the network after calculating loss for a single sample output compared to true output on that sample. 
- minibatch SGD: using minibatches (e.g. of 32, 64, 512) samples to calculate more 'precise' gradient direction. Is most commonly used technique


------------

2. Regarding SGD and GD:
  1. Provide at least two reasons for why SGD is used more often in practice compared to GD.
  2. In what cases can GD not be used at all?

------------

ANSWER:

As I understand, by SGD, the mini-batch SGD is meant. Pure SGD is not used a lot.

A. Why SGD is used more often than GD:

- The input set can be huge, and making a big calculation for a single step is costly.
- SGD calculates an approximation for gradient, and that's ok. The expectation of those gradients is the true gradient, and will lead to a minimum of the loss. 
We can control with the 'minibatch' size hyperparameter the amount of samples upon which this gradient averaging is made. Using this technique saves time and resources that we need to allocate if we work in parallel architectures.

B. GD will not be used at all if the input training set size is unlimited. 
For example, the inputs are some synthesized data (environment continous space in RL). We can synthesize any amount of data we want.


------------

3. You have trained a deep resnet to obtain SoTA results on ImageNet.
While training using mini-batch SGD with a batch size of $B$, you noticed that your model converged to a loss value of $l_0$ within $n$ iterations (batches across all epochs) on average.
Thanks to your amazing results, you secure funding for a new high-powered server with GPUs containing twice the amount of RAM.
You're now considering to increase the mini-batch size from $B$ to $2B$.
Would you expect the number of of iterations required to converge to $l_0$ to decrease or increase when using the new batch size? explain in detail.

------------

ANSWER:

In short on effect of the batch size:

The larger the batch size, the more accurate is the update to the weights that is calculated, because more examples are taken into account. With fewer examples, the gradient is more dependent on the specific examples that were used.

To answer the question:
The batch size has increased 2 times. The gradient updates are now more accurate, so i expect the convergence to the loss value $l_0$ to happpen with **less** than $n$ iterations. It should be noted that big batch sizes can lead to poor generalization. 



------------

4. For each of the following statements, state whether they're **true or false** and explain why.
  1. When training a neural network with SGD, every epoch we perform an optimization step for each sample in our dataset.
  1. Gradients obtained with SGD have less variance and lead to quicker convergence compared to GD.
  1. SGD is less likely to get stuck in local minima, compared to GD.
  1. Training  with SGD requires more memory than with GD.
  1. Assuming appropriate learning rates, SGD is guaranteed to converge to a local minimum, while GD is guaranteed to converge to the global minimum.
  1. Given a loss surface with a narrow ravine (high curvature in one direction): SGD with momentum will converge more quickly than Newton's method which doesn't have momentum.

------------

ANSWER:

 A. **TRUE**
 
 With SGD, each epoch we go over ALL of the samples in the training set, whether one by one or minibatches, and eventually perform optimization on each.
 
 B. **FALSE**
 
 First, gradients with SGD have bigger variance, because they take into calculation less samples. So it will take longer to converge.
 
 
 C. **TRUE**
 
Because we take into account each time some other minibatch. For a GD, the gradient for the whole dataset could be 0 at some point, while some minibatch (any permutation of the samples which compose a minibatch) will likely some gradient in this point. Or like in the tutorial 5 said, SGD will get a different loss surfact every iteration
 
 D. **FALSE**
 
 The whole point of using the SGD is to make a training feasible, when the input data sample size is big. Instead of computing the gradient on the error of the prediction of the network for the whole dataset, we compute each time the gradient only for some number (16, 32, etc) of samples.
 
 E. **FALSE**
 
 In general, nothing can guarantee that the GD will converge to the global minimum. This can only be guaranteed when there is only 1 minimum on the 'surface' of the cost function. Imagine that there are 2 minimums, and the current weights position the current loss on the slope of the local minimum. GD will inevitably follow the steepest gradient, which will lead to this local minimum.
 
 F. **TRUE** 
 
Momentum is an addition to the optimizer which accelerates the convergence. It uses the information of the last gradients (moving average) to accelerate the current update in the common direction. In the situation described above (narrow ravine), the momentum will be more useful, since it will accelerate the optimization in the direction along the ravine. While Newton's method will accelerate it more towards the ravine itself, which is not what we want. 

------------

5. In tutorial 5 we saw an example of bi-level optimization in the context of deep learning, by embedding an optimization problem as a layer in the network.
  **True or false**: In order to train such a network, the inner optimization problem must be solved with a descent based method (such as SGD, LBFGS, etc).
  Provide a mathematical justification for your answer.

------------

ANSWER:

**FALSE** 

This is not correct. In order to make a training possible, we need to find a way to transfer further the gradient with respect to the loss. Following the notations from the tutorial, we need to calculate:

$$
\delta \vec{z} =  \pderiv{\vec{y}}{\vec{z}} \delta\vec{y} =\mattr{R}\mat{K^{-1}}\delta\vec{y}.
$$

If the calculation of the Hessians ${K^{-1}}$ and ${R}$ is feasible, a closed form solution exists, and we can use it to propagate the gradient.

------------

6. You have trained a neural network, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$ for some arbitrary parametrized functions $f_l(\cdot;\vec{\theta}_l)$.
  Unfortunately while trying to break the record for the world's deepest network, you discover that you are unable to train your network with more than $L$ layers.
  1. Explain the concepts of "vanishing gradients", and "exploding gradients".
  2. How can each of these problems be caused by increased depth?
  3. Provide a numerical example demonstrating each.
  4. Assuming your problem is either of these, how can you tell which of them it is without looking at the gradient tensor(s)?

------------

ANSWER:

 A. I have already explained before, but here it is again:
 - Vanishing gradients - gradients are being multiplied as they progagate into the network during the backpropagation process. In sigmoid and tanh function the gradients are close to zero when the absolute value of the input is big. This will cause the deeper layers to get really small updates to their weights, thus not to learn.
 - Exploding gradients - are caused by the same idea of 'backpropagating' the gradients back in the network. It is caused by some weights being too large in absolute value, thus causing the $\frac{\delta L}{\delta W}$ ($L$ is loss) to be large, thus propagating backwards big losses, which are multiplied again by values bigger than 1 if the weights are large in the previous layer. 
 
 
 B. If the network is deep, the effect of multiplicating values, which are >> 1 or << 1 for numerous times increases exponentially. If the number of layers is big enough, we inevetably would have either one of 2 cases towards the root of the network.
 
 C. Basic examples. I will supply the ones given in the lecture: 
 - Vanishing Gradients
 
 The slope of the activation function will be << 1 consecutively for large amount of layers, e.g. 0.1. Then, let's say, for a network of 10 layers, the $\delta W_1$ would be $0.1^{9}$, which will be further multiplied by the learning rate when updating the weights
 - Exploding Gradients
 In contrast to the Vanishing gradients problem, the exploding gradients are caused rather by the weights in the network, and not the gradient slopes (Sigmoid and tanh max slope value is 1). 
 
 
 
 D. For vanishing gradients, we can spot that the network eventually stops learning, and will settle on a poor validation error. For exploding gradients, we will see large changes in the loss for each update. (we will see eventually values going to NaN - sign that they are too big in absolute value)
 
------------

### Backpropagation

1. You wish to train the following 2-layer MLP for a binary classification task:
  $$
  \hat{y} =\mat{W}_2~ \varphi(\mat{W}_1 \vec{x}+ \vec{b}_1) + \vec{b}_2
  $$
  Your wish to minimize the in-sample loss function is defined as
  $$
  L_{\mathcal{S}} = \frac{1}{N}\sum_{i=1}^{N}\ell(y,\hat{y}) + \frac{\lambda}{2}\left(\norm{\mat{W}_1}_F^2 + \norm{\mat{W}_2}_F^2 \right)
  $$
  Where the pointwise loss is binary cross-entropy:
  $$
  \ell(y, \hat{y}) =  - y \log(\hat{y}) - (1-y) \log(1-\hat{y})
  $$
  
  Write an analytic expression for the derivative of the final loss $L_{\mathcal{S}}$ w.r.t. each of the following tensors: $\mat{W}_1$, $\mat{W}_2$, $\mat{b}_1$, $\mat{b}_2$, $\mat{x}$.

------------

ANSWER:

Following the tutorial 3, we rewrite the model:

$$
\hat{y} =\mat{W}_2~ \underbrace{\varphi(\overbrace{\mat{W}_1 \vec{x}+ \vec{b}_1}^{\vec{z}})}_{\vec{a}} + \vec{b}_2,
$$

The loss function here is the binary cross-entropy. Calculating the derivative of the loss w.r. to the prediction:

$$\pderiv{\ell}{\vec{\hat y}} = -{\vec y} \cdot \frac{1}{\vec{\hat y}} + (1-\vec{\hat{y}}) \cdot \frac{1}{1-\vec{\hat{y}}} = ... = \frac {\vec{\hat{y}} - {\vec{\hat{y}}}}{\vec{\hat{y}}\cdot(1-\vec{\hat{y}})}$$

Calculating for each of the required variables using the chain rule:


1. For $\mat{W}_2$:

$$
\pderiv{\ell}{\mat{W}_2}= \pderiv{\ell}{\vec{\hat y}}\pderiv{\vec{\hat y}}{\mat{W}_2}
=\left( \frac {\vec{\hat{y}} - {\vec{\hat{y}}}}{\vec{\hat{y}}\cdot(1-\vec{\hat{y}})} \right) \vectr{a}
$$
$$
\nabla_{\mat{W}_2}L_{\mathcal{S}}=\frac{1}{N}\sum_{i=1}^{N} \pderiv{\ell_i}{\mat{W}_2} + \lambda\mat{W}_2
$$

2. For $\mat{W}_1$:

We need to use the chain rule further to calculate its derivatives:

$$
\pderiv{\ell}{\mat{W}_1}= \pderiv{\ell}{\vec{\hat y}}\pderiv{\vec{\hat y}}{\mat{a}}\pderiv{\mat{a}}{\mat{z}}\pderiv{\mat{z}}{\mat{W}_1} = \pderiv{\ell}{\vec{\hat y}}\mat{W}_2^T\pderiv{\mat{a}}{\mat{z}}\mat{x}^{T}
$$
$$
\nabla_{\mat{W}_1}L_{\mathcal{S}}=\frac{1}{N}\sum_{i=1}^{N} \pderiv{\ell_i}{\mat{W}_1} + \lambda\mat{W}_1
$$
where we do not know $\pderiv{\mat{a}}{\mat{z}}$ because we are not given the activation function.

And $\pderiv{\ell}{\vec{\hat y}}$ is given above


3. For $\vec{b}_2$:

$$
\pderiv{\ell}{\vec{b}_2}= \pderiv{\ell}{\vec{\hat y}}\pderiv{\vec{\hat y}}{\vec{b}_2} = \pderiv{\ell}{\vec{\hat y}}
$$
$$
\nabla_{\vec{b}_2}L_{\mathcal{S}}=\frac{1}{N}\sum_{i=1}^{N} \pderiv{\ell_i}{\vec{b}_2}
$$


4. For $\vec{b}_1$:

We need to use the chain rule further to calculate its derivatives:

$$
\pderiv{\ell}{\vec{b}_1}= \pderiv{\ell}{\vec{\hat y}}\pderiv{\vec{\hat y}}{\mat{a}}\pderiv{\mat{a}}{\mat{z}}\pderiv{\mat{z}}{\vec{b}_1} = \pderiv{\ell}{\vec{\hat y}}\mat{W}_2^T\pderiv{\mat{a}}{\mat{z}}
$$
$$
\nabla_{\mat{W}_1}L_{\mathcal{S}}=\frac{1}{N}\sum_{i=1}^{N} \pderiv{\ell_i}{\mat{W}_1}
$$
where we do not know $\pderiv{\mat{a}}{\mat{z}}$ because we are not given the activation function

And $\pderiv{\ell}{\vec{\hat y}}$ is given above


5. For $\mat{x}$:

We need to use the chain rule further to calculate its derivatives:

$$
\pderiv{\ell}{\mat{x}}= \pderiv{\ell}{\vec{\hat y}}\pderiv{\vec{\hat y}}{\mat{a}}\pderiv{\mat{a}}{\mat{z}}\pderiv{\mat{z}}{\mat{x}} = \pderiv{\ell}{\vec{\hat y}}\mat{W}_2^T\pderiv{\mat{a}}{\mat{z}}\mat{W}_1^T
$$
$$
\nabla_{\mat{x}}L_{\mathcal{S}}=\frac{1}{N}\sum_{i=1}^{N} \pderiv{\ell_i}{\mat{x}}
$$
where we do not know $\pderiv{\mat{a}}{\mat{z}}$ because we are not given the activation function.

And $\pderiv{\ell}{\vec{\hat y}}$ is given above

------------

2. Derivative a function $f(\vec{x})$ at a point $\vec{x}_0$ is
  $$
  f'(\vec{x}_0)=\lim_{\Delta\vec{x}\to 0} \frac{f(\vec{x}_0+\Delta\vec{x})-f(\vec{x}_0)}{\Delta\vec{x}}
  $$
  
  1. Explain how this formula can be used in order to compute gradients of neural network parameters numerically, without automatic differentiation (AD).
  
  2. What are the drawbacks of this approach? List at least two drawbacks compared to AD.

------------

ANSWER:

A. As the equation suggests, we can calculate the gradient at some specific point $x_0$. By choosing some small $\epsilon$, we can numerically obtain the gradient.

B. Some drawbacks of numerical gradient calculation:

- It can give rounding errors due to the discretization of $\epsilon$.
- It is much slower to calculate partial derivatives this way. Partial derivatives are what we need for the gradient propagation though the net.
- The choice of $\epsilon$ introduces another hyperparameter which has to be tuned...



------------

3. Given the following code snippet:
  1. Write a short snippet that implements that calculates gradient of `loss` w.r.t. `W` and `b` using the approach of numerical gradients from the previous question.
  2. Calculate the same derivatives with autograd.
  3. Show, by calling `torch.allclose()` that your numerical gradient is close to autograd's gradient.

In [4]:
import torch

N, d = 100, 5
dtype = torch.float64
X = torch.rand(N, d, dtype=dtype)
W, b = torch.rand(d, d, requires_grad=True, dtype=dtype), torch.rand(d, requires_grad=True, dtype=dtype)

def foo(W, b):
    return torch.mean(X @ W + b)

loss = foo(W, b)
print(f"{loss=}")

# TODO: Calculate gradients numerically for W and b
grad_W = torch.zeros_like(W)
grad_b = torch.zeros_like(b)

# we need to choose some epsilon
eps = 1e-5
W_clone = torch.clone(W)
b_clone = torch.clone(b)

# calculate for W - calculate the loss derivative compared to each of the weights independently.
for i in range(W.shape[0]):
    for j in range(W.shape[1]):

        # modify one item
        W_clone[i,j] = W_clone[i,j] + eps
        
        # calculate loss with this change
        new_loss = foo(W_clone, b)
        
        # calculate the numerical gradient
        grad_W[i, j] = (new_loss - loss) / (eps)
        
        # modify the item back
        W_clone[i,j] = W_clone[i,j] - eps     
        
# calculate for b - calculate the loss derivative compared to each of the weights independently.
for i in range(b.shape[0]):

    # modify one item
    b_clone[i] = b_clone[i] + eps

    # calculate loss with this change
    new_loss = foo(W, b_clone)

    # calculate the numerical gradient
    grad_b[i] = (new_loss - loss) / (eps)

    # modify the item back
    b_clone[i] = b_clone[i] - eps         


# TODO: Compare with autograd using torch.allclose()
loss.backward() # added loss calculation
autograd_W = W.grad
autograd_b = b.grad

assert torch.allclose(grad_W, autograd_W)
assert torch.allclose(grad_b, autograd_b)

loss=tensor(1.8000, dtype=torch.float64, grad_fn=<MeanBackward0>)


### Sequence models

1. Regarding word embeddings:
  1. Explain this term and why it's used in the context of a language model.
  1. Can a language model like the sentiment analysis example from the tutorials be trained without an embedding? If yes, what would be the consequence for the trained model? if no, why not?

------------

ANSWER:

A. Word embeddings are 'vectorized' form of the word, used to represent them in a low dimensional space. It allows to measure similarity between words (e.g. as a simple Euclidean distance), to predict the words around a certain word, or to capture inter-word semantics.

B. Without embedding means "one-hot encoding". In this method, no relations between words in a sentence can be captured, and no similarity can be measured (it can, but will be $\sqrt{2}$ for different words, or 0 for same words, since only 1 item in a vector is 1, others are 0). In general, the model can be learned, but it will have very poor performance and unable to capture semantics in the sentence.

------------

2. Considering the following snippet, explain:
  1. What does `Y` contain? why this output shape?
  2. How you would implement `nn.Embedding` yourself using only torch tensors. 

In [5]:
import torch.nn as nn

X = torch.randint(low=0, high=42, size=(5, 6, 7, 8))
embedding = nn.Embedding(num_embeddings=42, embedding_dim=42000)
Y = embedding(X)
print(f"{Y.shape=}")

Y.shape=torch.Size([5, 6, 7, 8, 42000])


------------

ANSWER:

A. 

Y contains the embedding of each of the elements in X. Each element in X is in range of 0 to 41 (dictionary size). nn.Embedding is a LUT which generates and stores embedding for each of the dictionary values. E.g. if values at certain indexes at X have same value (e.g. 32), they will have same embedding in corresponding indexes in Y.

B. 

The piece of code is attached below. Since the embedding is a simple LUT, we simply create a Tensor which serves as LUT of size [dict size X embedding dim]. I did not try to make this code effective, rather understandable.

------------

In [6]:
seed = 22
torch.manual_seed(seed)

X = torch.randint(low=0, high=42, size=(2, 2, 3, 8))

embedding_layer = torch.rand(42, 42000) # can also be different initialization
Y2 = X.clone()


Y2 = Y2.unsqueeze(dim=4)
Y2 = Y2.repeat(1,1,1,1,42000)
Y2 = Y2.type(torch.DoubleTensor)


for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        for m in range(X.shape[2]):
            for n in range(X.shape[3]):
                
                Y2[i,j,m,n] = embedding_layer[X[i,j,m,n]]
                

# those are same dict values                
print(X[0,0,0][1])
print(X[0,0,0][4])

# we expect same embeddings
print(Y2[0,0,0][1][0:20])
print(Y2[0,0,0][4][0:20])


tensor(28)
tensor(28)
tensor([0.4805, 0.8882, 0.6731, 0.3894, 0.1991, 0.5775, 0.8485, 0.9828, 0.9393,
        0.0321, 0.9216, 0.1930, 0.0304, 0.0444, 0.3972, 0.5821, 0.8852, 0.5962,
        0.6283, 0.8778], dtype=torch.float64)
tensor([0.4805, 0.8882, 0.6731, 0.3894, 0.1991, 0.5775, 0.8485, 0.9828, 0.9393,
        0.0321, 0.9216, 0.1930, 0.0304, 0.0444, 0.3972, 0.5821, 0.8852, 0.5962,
        0.6283, 0.8778], dtype=torch.float64)


3. Regarding truncated backpropagation through time (TBPTT) with a sequence length of S: State whether the following sentences are **true or false**, and explain.
  1. TBPTT uses a modified version of the backpropagation algorithm.
  2. To implement TBPTT we only need to limit the length of the sequence provided to the model to length S.
  3. TBPTT allows the model to learn relations between input that are at most S timesteps apart.

------------

ANSWER:

A. **TRUE**

TBPTT is a variant of BPTT

B. **FALSE**

In the forward pass, we don't truncate. We only limit the sequence when using the backpropagation. It helps to reduce complexity of the model, and noise (inputs from far away in the past won't affect much the present). One of drawbacks is that the model will become more short-term focused (depends on S, of course).

C. **TRUE**

As explained in B, the model becomes more short-term focused, and any inputs further away than S timesteps will not have effect on the current timestamp gradients calculations.

------------

### Attention

1. In tutorial 7 (part 2) we learned how to use attention to perform alignment between a source and target sequence in machine translation.
  1. Explain qualitatively what the addition of the attention mechanism between the encoder and decoder does to the hidden states that the encoder and decoder each learn to generate (for their language). How are these hidden states different from the model without attention
  
  2. After learning that self-attention is gaining popularity thanks to the shiny new transformer models, you decide to change the model from the tutorial: instead of the queries being equal to the decoder hidden states, you use self-attention, so that the keys, queries and values are all equal to the encoder's hidden states (with learned projections). What influence do you expect this will have on the learned hidden states?


------------

ANSWER:

A. With the addition of the Attention mechanism, the encoder and decoder hidden states learn to generate different things. 

The encoder without attention had to learn  to encode the entire meaning into the latent representation $z$. The decoder then tries to retrieve the whole target sequence from this latent representation.

With attention, the encoder hidden state now learns to encode the meaning of each of the words, additionally to the connection between them. The decoder learns to take into the account the similarity vector between the query (each new output) and the key-value pairs from the encoder outputs. Meaning it has an ability to look into the local information of the encoder, and not only global.


B. The learned hidden states in the encoder will also have to incorporate the attention knowledge regarding the previous inputs in the sequence. (but not next inputs). It allows the inputs to interact with each other.


------------

### Unsupervised learning

1. As we have seen, a variational autoencoder's loss is comprised of a reconstruction term and  a KL-divergence term. While training your VAE, you accidentally forgot to include the KL-divergence term.
What would be the qualitative effect of this on:

  1. Images reconstructed by the model during training ($x\to z \to x'$)?
  1. Images generated by the model ($z \to x'$)?

------------

ANSWER:

The KL-divergence term aims to 'model' the input data in the latent space. in VAE, by using the Bayesian methods to represent the data as a continuous distribution in the latent space, it allows to generate new data by sampling from the latent space and using the decoder.

A. Without the KL-divergence, the images reconstruction will not suffer. Since we don't care about the latent space, and with the network sophisticated enough we can perfectly encode the input samples.

B. Without the KL-divergence, the generation of the new samples will suffer. Because the latent space it not continuous, the similarity of samples in the $x$ space does not imply similarity in the $z$ space, there will be gaps in the latent representations, etc. 


------------

2. Regarding VAEs, state whether each of the following statements is **true or false**, and explain:
  1. The latent-space distribution generated by the model for a specific input image is $\mathcal{N}(\vec{0},\vec{I})$.
  2. Every time we feed an image to the encoder, then decode the result, we'll get the same reconstruction.
  3. Since the real VAE loss term is intractable, what we actually minimize instead is it's upper bound, in the hope that the bound is tight.

------------

ANSWER:

A. **FALSE**

The distribution $\mathcal{N}(\vec{0},\vec{I})$ is the distribution used in the Loss function in the KL loss term. Means, we aim to map the input space to this distribution, but it does not mean that this is what will happen. There is a trade-off between the reconstruction and the KL-divergence terms in the Loss function.

B. **FALSE**

Each time we feed an input $x$, we sample $z$ from the approximation in the latent space $q(z|x)$. This latent vector is then fed into the decoder, which projects the latent space vector back to the input space. So no, in VAE, each time the reconstruction will be different. (unless by chance we sample the same $z$... )

C. **TRUE**

Correct. We use the ELBO - evidence lower bound. (maybe it's a typo in a question - but we **maximize** the lower bound)

------------


3. Regarding GANs, state whether each of the following statements is **true or false**, and explain:
  1. Ideally, we want the generator's loss to be low, and the discriminator's loss to be high so that it's fooled well by the generator.
  2. It's crucial to backpropagate into the generator when training the discriminator.
  3. To generate a new image, we can sample a latent-space vector from $\mathcal{N}(\vec{0},\vec{I})$.
  4. It can be beneficial for training the generator if the discriminator is trained for a few epochs first, so that it's output isn't arbitrary.
  5. If the generator is generating plausible images and the discriminator reaches a stable state where it has 50% accuracy (for both image types), training the generator more will further improve the generated images.

------------

ANSWER:

A. **TRUE**.  The main goal of the GAN is to train the generator, not the discriminator. If the generator performs so well that the discriminators accuracy is 1/2, then our task is complete. 

B. **FALSE**. We train them separately. 

C. **TRUE**. Although this is not limited to this distribution only - we can train the generator with any latent space distribution. 

D. **FALSE**. The discriminator needs some fake data too in order to be trained properly to make the classes (real & fake) represented equally. When all of its inputs will be the 'real' data, the discriminator will learn to output 'real' for any input it receives. 

E. **FALSE**. When the discriminator has 50% accuracy, this means it cannot distinguish the real and the fake data. Meaning that it makes a random guess for the image. Since this guess is random, it has no meaning for training the generator, and will not improve its performance


------------

### Graph Neural Networks

1. You have implemented a graph convolutional layer based on the following formula, for a graph with $N$ nodes:
$$
\mat{Y}=\varphi\left( \sum_{k=1}^{q} \mat{\Delta}^k \mat{X} \mat{\alpha}_k + \vec{b} \right).
$$
  1. Assuming $\mat{X}$ is the input feature matrix of shape $(N, M)$: what does $\mat{Y}$ contain in it's rows?
  1. Unfortunately, due to a bug in your calculation of the Laplacian matrix, you accidentally zeroed the row and column $i=j=5$ (assume more than 5 nodes in the graph).
What would be the effect of this bug on the output of your layer, $\mat{Y}$?

------------

ANSWER:

A. the $Y$ is in the shape of $(N, M')$, where the $M'$ came from the dimension of the weights $a_k (M,M')$.

Each row of the $Y$ matrix contains resulting features for each vertex from the operation of the graph convolution and activation function on the verteces features from the previous layer. Graph convolution in this case is application of the averaging (multiplication with different functions on the Laplacians - in this case polynomials) and combination across averaging using the weights matrixes $a_k$. Plus some bias term.


B. Laplacian contains the graph information. First of all, all the information of relation of vertex 5 with other verteces is lost. Secondly, we can see from the calculations that the 5'th row of the $Y$ matrix will also contain only zeros. 

------------

2. We have discussed the notion of a Receptive Field in the context of a CNN. How would you define a similar concept in the context of a GCN (i.e. a model comprised of multiple graph convolutional layers)?

------------

ANSWER:

In the concept of GCN this would be the number of verteces that affect the output features for a certain vertex.

------------